In [1]:
import pandas as pd
import datetime

In [2]:
actors = pd.read_csv('../Tables/Actors.csv')
directors = pd.read_csv('../Tables/Directors.csv')
genres = pd.read_csv('../Tables/Genres.csv')
movies = pd.read_csv('../Tables/Movie_Data.csv')
studios = pd.read_csv('../Tables/Studios.csv')
titles = pd.read_csv('../Tables/titleNid.csv')

In [3]:
actor_mean = actors.groupby('movie_id', as_index=False, sort = False).mean()
actor_mean = actor_mean.drop(['actor_id','gender'],axis=1)
actor_mean.to_csv('../Tables/actor_mean.csv', index=False)

In [14]:
director_mean = directors.groupby('movie_id', as_index=False , sort = False).mean()
director_mean = director_mean.drop(['director_id','gender'],axis=1)
director_mean.to_csv('../Tables/director_mean.csv', index=False)
director_mean.count()


movie_id               1905
director_popularity    1905
dtype: int64

In [5]:
count = studios.studio_name.value_counts()
replace = list(count[count <= 3].index)
replace
for studio in replace:
    studios.studio_name = studios.studio_name.replace(studio, "Other")
grouped_df = studios.groupby('movie_id',sort = False)
grouped_lists = grouped_df['studio_name'].agg(lambda column: ", ".join(column))
grouped_lists = grouped_lists.reset_index(name='studio_name')
holder = []
for names in grouped_lists['studio_name']:
    holder.append(names.split(","))
# movies['studio_list'] = holder
# movies
grouped_lists['studio_list'] = holder
grouped_lists = grouped_lists.drop('studio_name', 1)
grouped_lists.to_csv('../Tables/studio_assm.csv', index=False)


In [6]:
movies['studios'] = grouped_lists['studio_list']
movies['release_date'] = pd.to_datetime(movies['release_date'], format= '%Y-%m-%dT')
genres.set_index(genres['id'],inplace = True)
genres = genres.drop('id',1)

genre_title = []

for ls in movies.genre_ids:
    holder = []
    for id in eval(ls):
        holder.append(genres.loc[id]['name'])
    genre_title.append(holder)

movies['genres'] = genre_title
movies = movies.drop('genre_ids',1)
movies.to_csv('../Tables/movies_clean.csv', index=False)

In [8]:
ml_table = movies[['movie_id','release_date','movie_popularity', 'vote_average', 'vote_count','budget','revenue', 'rating', 'genres','studios']]
ml_table = ml_table.merge(actor_mean,on='movie_id', how = 'inner')
ml_table = ml_table.merge(director_mean,on='movie_id', how = 'inner')
ml_table = ml_table.drop('movie_id',1)
ml_table

,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating,genres,studios,actor_popularity,director_popularity
0,2021-09-30,13071.277,7.2,3726,110000000,454000000,PG-13,"[Science Fiction, Action, Adventure]","[Marvel Entertainment, Pascal Pictures, Colu...",4.182811,11.123
1,2021-09-01,3517.554,7.9,3848,150000000,430238384,PG-13,"[Action, Adventure, Fantasy]",[Marvel Studios],5.082263,1.830
2,2021-09-29,1938.906,7.6,2283,242000000,758000000,PG-13,"[Adventure, Action, Thriller]","[Eon Productions, Metro-Goldwyn-Mayer, Unite...",3.767727,3.413
3,2021-11-03,1339.598,7.1,1293,200000000,368000000,PG-13,"[Action, Adventure, Science Fiction, Drama]",[Marvel Studios],4.528703,2.558
4,2021-08-11,1246.828,7.8,3909,110000000,331096766,PG-13,"[Comedy, Action, Adventure, Science Fiction]","[Other, 21 Laps Entertainment, Other, Other]",3.894072,5.383
...,...,...,...,...,...,...,...,...,...,...,...
1900,2008-07-08,18.426,5.3,1092,60000000,50650079,PG,"[Comedy, Science Fiction, Adventure, Family]","[Dune Entertainment, Regency Enterprises, Ot...",2.892000,1.308
1901,2016-02-19,18.418,5.9,1710,20000000,12639297,R,"[Action, Thriller, Mystery]","[Other, Worldview Entertainment, Other, Oth...",2.836000,1.380
1902,1995-04-21,18.401,7.4,1424,2000000,2381087,R,"[Drama, Crime]","[New Line Cinema, Other]",6.069867,0.600
1903,1985-06-29,18.401,6.2,2084,10000000,36230219,PG-13,"[Action, Adventure, Science Fiction]",[Kennedy Miller Productions],1.566789,1.871


release_date           datetime64[ns]
movie_popularity              float64
vote_average                  float64
vote_count                      int64
budget                          int64
revenue                         int64
rating                         object
genres                         object
studios                        object
actor_popularity              float64
director_popularity           float64
dtype: object